In [4]:
import sys, os
from Scripts import train_metric_learning, run_metric_learning_inference, train_gnn, run_gnn_inference, build_track_candidates
import yaml

import torch 
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'{device} available')
import pandas as pd, numpy as np


from bokeh.io import output_notebook, show
output_notebook()
from bokeh.plotting import figure, row
from bokeh.models import ColumnDataSource
from bokeh.palettes import viridis

from Pipelines.Common_Tracking_Example.notebooks.ITk.Exploration.gnn_utils import infer_event
from Pipelines.TrackML_Example.notebooks.build_embedding import EmbeddingInferenceBuilder

from matplotlib import pyplot as plt

from pytorch_lightning import Trainer

from IPython.display import clear_output
CONFIG = 'pipeline_config.yaml'

cuda available


Loading BokehJS ...

# 0. Download Data

In [ ]:
!mkdir datasets
!wget https://portal.nersc.gov/cfs/m3443/dtmurnane/TrackML_Example/trackml_quickstart_dataset.tar.gz -O datasets/trackml_quickstart_dataset.tar.gz
!tar -xvf datasets/trackml_quickstart_dataset.tar.gz -C datasets

### Pipeline configurations

The configurations for the entire pipeline are defined under pipeline_config.yml. 

In [2]:
with open(CONFIG, 'r') as f:
    configs = yaml.load(f, Loader=yaml.FullLoader)
print(yaml.dump(configs))

common_configs:
  artifact_directory: artifacts
  experiment_name: trackml_quickstart_1
  gpus: 1
  max_epochs: 20
gnn_configs:
  aggregation: sum_max
  cell_channels: 8
  datatype_names:
  - train
  - val
  - test
  datatype_split:
  - 80
  - 10
  - 10
  edge_cut: 0.5
  factor: 0.3
  hidden: 128
  hidden_activation: SiLU
  input_dir: datasets/quickstart_metric_learning_processed
  layernorm: true
  lr: 0.001
  mask_background: true
  n_graph_iters: 8
  nb_edge_layer: 3
  nb_node_layer: 3
  noise: false
  output_dir: datasets/quickstart_gnn_processed
  patience: 10
  pt_background_min: 1.0
  pt_signal_min: 1.0
  regime:
  - - pid
  spatial_channels: 3
  truth_key: pid_signal
  warmup: 5
  weight: 2
metric_learning_configs:
  activation: Tanh
  cell_channels: 9
  emb_dim: 12
  emb_hidden: 1024
  factor: 0.5
  input_dir: datasets/quickstart_example_1GeV
  knn: 100
  lr: 0.001
  margin: 0.1
  nb_layer: 4
  output_dir: datasets/quickstart_metric_learning_processed
  patience: 30
  points_p

# 1. Train Metric Learning

## What it does
Broadly speaking, the first stage of our pipeline is embedding the space points on to graphs, in a way that is efficient, i.e. we miss as few points on a graph as possible. We train a MLP to transform the input feature vector of each space point $\mathbf{u}_i$ into an N-dimensional latent space $\mathbf{v}_i$. The graph is then constructed by connecting the space points whose Euclidean distance between the latent space points $$d_{ij} = \left| \mathbf{v}_i - \mathbf{v}_j \right| < r_{embedding}$$

## Training data
Let us take a look at the data before training. In this example pipeline, we have preprocessed the TrackML data into a more convenient form. We calculated directional information and summary statistics from the charge deposited in each spacepoints, and append them to its cyclidrical coordinates. Let us load an example data file and inspect the content.

In [5]:
from Pipelines.TrackML_Example.LightningModules.Embedding.Models.layerless_embedding import LayerlessEmbedding

metric_learning_configs = configs['metric_learning_configs']

model = LayerlessEmbedding(metric_learning_configs)
model.setup(stage='fit')
clear_output()

print(model.trainset[0])
example_data = model.get_input_data(model.trainset[0])
example_data_df = pd.DataFrame(example_data.numpy())
example_data_df.head()

Data(x=[12083, 3], pid=[12083], modules=[12083], event_file='datasets/quickstart_example_1GeV/21045', hid=[12083], pt=[12083], weights=[10965], modulewise_true_edges=[2, 10965], layerwise_true_edges=[2, 14426], cell_data=[12083, 9], signal_true_edges=[2, 10965])


,0,1,2,3,4,5,6,7,8,9,10,11
0,1.0,0.323412,2.091356,0.844154,0.05,0.05625,0.3,-2.091356,0.962261,0.051929,0.083736,-0.958000
1,6.0,0.308704,0.884925,1.229181,0.10,0.28125,0.3,0.800960,1.972132,0.115441,0.501320,-0.198762
2,6.0,0.312759,0.793395,1.423718,0.05,0.33750,0.3,0.956851,2.072294,0.031444,0.612759,0.041935
3,7.0,0.342820,0.772962,1.282741,0.10,0.33750,0.3,0.928149,-0.127298,0.031484,-0.159847,-0.085926
4,3.0,0.162364,1.440542,0.844154,0.10,0.11250,0.3,0.348650,2.327071,0.071830,0.609832,-0.018804


The input data is gotten by concatenating the cell data and cylindrical coordinate of each space point 

In [7]:
input_data = [model.trainset[0].cell_data.numpy(), model.trainset[0].x.numpy()]
input_data = np.concatenate(input_data, axis=1)
input_data = pd.DataFrame(input_data)
input_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.0,0.323412,2.091356,0.844154,0.05,0.05625,0.3,-2.091356,0.962261,0.051929,0.083736,-0.958000
1,6.0,0.308704,0.884925,1.229181,0.10,0.28125,0.3,0.800960,1.972132,0.115441,0.501320,-0.198762
2,6.0,0.312759,0.793395,1.423718,0.05,0.33750,0.3,0.956851,2.072294,0.031444,0.612759,0.041935
3,7.0,0.342820,0.772962,1.282741,0.10,0.33750,0.3,0.928149,-0.127298,0.031484,-0.159847,-0.085926
4,3.0,0.162364,1.440542,0.844154,0.10,0.11250,0.3,0.348650,2.327071,0.071830,0.609832,-0.018804


In [8]:
with torch.no_grad():
    latent = model(example_data)

latent_df = pd.DataFrame(latent.numpy())
latent_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,-0.053065,0.197346,0.262761,0.384845,-0.557306,-0.200907,-0.290017,-0.038574,-0.108788,-0.086592,-0.162083,-0.508856
1,0.454764,-0.016605,0.018029,0.318143,0.442434,-0.060065,0.056463,0.570523,0.214677,0.245423,0.033910,-0.236358
2,0.481569,-0.004230,0.017104,0.316161,0.450244,-0.035352,0.024962,0.552444,0.203532,0.238946,0.019835,-0.243150
3,0.366191,-0.028428,0.035583,0.390730,0.393709,-0.084740,0.204175,0.483061,-0.027342,0.352563,0.074680,-0.378564
4,0.535915,0.100143,-0.021422,0.323075,0.432406,0.093268,-0.138293,0.518372,0.178168,-0.271230,-0.032768,-0.089656


## Train metric learning model

Finally we come to model training. By default, we train the MLP for 20 epochs, which amounts to approximately 15 minutes. Feel free to adjust the epoch number in pipeline_config.yml

In [9]:
metric_learning_trainer, metric_learning_model = train_metric_learning(CONFIG)
clear_output()
log_file = os.path.join(metric_learning_trainer.logger.log_dir , 'metrics.csv')

If you have pretrained a model, load it by 

In [27]:
from Pipelines.TrackML_Example.LightningModules.Embedding.Models.layerless_embedding import LayerlessEmbedding
# copy/paste training checkpoint path here
METRIC_LEARNING_CKPT_PATH = 'artifacts/metric_learning/trackml_quickstart_1/version_34/checkpoints/epoch=19-step=1600.ckpt'
metric_learning_model = LayerlessEmbedding.load_from_checkpoint( checkpoint_path= METRIC_LEARNING_CKPT_PATH)
log_file = os.path.join( os.path.split(os.path.split(METRIC_LEARNING_CKPT_PATH)[0])[0] , 'metrics.csv')
metric_learning_model.setup(stage='fit')
clear_output()

## Plot training metrics

In [26]:
# log_file = os.path.join(metric_learning_trainer.logger.log_dir , 'metrics.csv')
metrics = pd.read_csv(log_file, sep=',')
train_metrics = metrics[ ~ metrics['train_loss'].isna() ][['epoch', 'train_loss']]
train_metrics['epoch'] -= 1
val_metrics = metrics[ ~ metrics['val_loss'].isna() ][['val_loss', 'eff', 'pur', 'current_lr', 'epoch']]
metrics = pd.merge(left=train_metrics, right=val_metrics, how='inner', on='epoch')
metrics.head()

,epoch,train_loss,val_loss,eff,pur,current_lr
0,0,0.012738,0.009860,0.716931,0.008744,0.0002
1,1,0.009994,0.009851,0.802390,0.009786,0.0004
2,2,0.009975,0.009740,0.860936,0.010500,0.0006
3,3,0.010033,0.009849,0.697635,0.008508,0.0008
4,4,0.009987,0.009822,0.832151,0.010149,0.0010


In [6]:
p1 = figure(title='Training validation loss', x_axis_label='Epoch', y_axis_label='Loss')

source = ColumnDataSource(metrics)

cmap = viridis(3)

for idx, y in enumerate(['train_loss', 'val_loss']):
    p1.circle(y=y, x='epoch', source=source, color=cmap[idx], legend_label=y)
    p1.line(x='epoch', y=y, source=source, color=cmap[idx], legend_label=y)


p2 = figure(title='Purity on validation set', x_axis_label='Epoch', y_axis_label='Purity')
p2.circle(y='pur', x='epoch', source=source, color=cmap[0], legend_label='Purity')
p2.line(x='epoch', y='pur', source=source, color=cmap[0], legend_label='Purity')

p3 = figure(title='Efficiency on validation set', x_axis_label='Epoch', y_axis_label='Efficiency')
p3.circle(y='eff', x='epoch', source=source, color=cmap[0], legend_label='Efficiency')
p3.line(x='epoch', y='eff', source=source, color=cmap[0], legend_label='Efficiency')

show(row([p1,p2, p3]))

## Evaluate model performance on sample test data

Here we evaluate the model performace on one sample test data. We look at how the efficiency and purity change with the embedding radius.

In [28]:
all_efficiencies, all_purities, all_losses = [], [], []
all_radius = np.arange(0.001, 0.4, 0.005)
results = { 'eff': [], 'pur': [], 'loss': [], 'radius': all_radius }
metric_learning_model.to(device)
test_data = metric_learning_model.testset[0].to(device)

with torch.no_grad():
    for r in all_radius:
        test_results = metric_learning_model.shared_evaluation(
            test_data, 0, r, configs['metric_learning_configs']['knn'], log=False
        )
        for key in results:
            if key not in test_results: continue
            results[key].append( test_results[key].cpu().numpy() )
results = pd.DataFrame(results)

source = ColumnDataSource(results)
cmap = viridis(3)
titles = ['Efficiency', 'Purity', 'Loss'] 
figures = []
x='radius'
for idx, y in enumerate(['eff', 'pur', 'loss']):
    figures.append( figure(title=titles[idx], x_axis_label=x, y_axis_label=y) )
    figures[-1].circle(y=y, x=x, source=source, color=cmap[0], legend_label=y)
    figures[-1].line(x=x, y=y, source=source, color=cmap[0], legend_label=y)

show(row(figures))

## Plot example truth and predicted graphs

In [64]:
# from matplotlib import pyplot as plt
test_results = metric_learning_model.to(device).shared_evaluation(test_data.to(device), 0, 0.15, configs['metric_learning_configs']['knn'], log=False)

p = figure(title='Truth graphs', x_axis_label='x', y_axis_label='y', height=800, width=800) 
q = figure(title='Predicted graphs', x_axis_label='x', y_axis_label='y', height=800, width=800) 

signal_true_edges = test_data.signal_true_edges
pred_edges = test_results['preds']
pid = test_data.pid
r, phi, z = test_data.cpu().x.T
x, y = r * np.cos(phi * np.pi), r * np.sin(phi * np.pi)
i=0
j=0
cmap = viridis(11)
source = ColumnDataSource(dict(x=x.numpy(), y=y.numpy()))
p.circle(x='x', y='y', source=source, color=cmap[0], size=1, alpha=0.1)
q.circle(x='x', y='y', source=source, color=cmap[0], size=1, alpha=0.1)

while i < 10:
    j += 1
    track = test_data.pid.unique()[j]
    e = signal_true_edges[:, pid[ signal_true_edges[0]] == track ]
    # true_edges = pid[ e[0]] == pid[e[1]]
    # plt.plot(x[e[:, ~true_edges]], y[e[:, ~true_edges]], c="r")
    # X, Y = x[e[:, true_edges]][0].numpy(), y[e[:, true_edges]][0].numpy()
    # X1, Y1, X2, Y2 = x[e[:, true_edges]].numpy()[0], y[e[:, true_edges]].numpy()[0], x[e[:, true_edges]].numpy()[1], y[e[:, true_edges]].numpy()[1]
    X1, Y1, X2, Y2 = x[e].numpy()[0], y[e[:]].numpy()[0], x[e[:]].numpy()[1], y[e[:]].numpy()[1]
    if not X1.shape[0] > 10: continue
    X = np.append(X1, X2[-1])
    Y = np.append(Y1, Y2[-1])

    source = ColumnDataSource(dict(x=X, y=Y))
    i += 1

    p.circle(x='x', y='y', source=source, color=cmap[i], size=5)
    p.line(x='x', y='y', source=source, line_color=cmap[i], line_width=1)

    e = pred_edges[:, (pid[ pred_edges[0]] == track) | (pid[ pred_edges[1]] == track) ]
    true_edges = pid[ e[0]] == pid[e[1]]

    # X1, Y1, X2, Y2 = x[e[:, true_edges]].numpy()[0], y[e[:, true_edges]].numpy()[0], x[e[:, true_edges]].numpy()[1], y[e[:, true_edges]].numpy()[1]
    X1, Y1, X2, Y2 = x[e].numpy()[0], y[e[:]].numpy()[0], x[e[:]].numpy()[1], y[e[:]].numpy()[1]
    # if not X1.shape[0] > 10: continue
    X = np.append(X1, X2[-1])
    Y = np.append(Y1, Y2[-1])

    source = ColumnDataSource(dict(x=X, y=Y))

    q.circle(x='x', y='y', source=source, color=cmap[i], size=5)
    q.line(x='x', y='y', source=source, line_color=cmap[i], line_width=1)
    


show(row([p,q]))

## Histogram of graph size

In [69]:
all_true_edges = []
all_pred_edges = []
signal_true_edges = test_data.signal_true_edges
pred_edges = test_results['preds']
pid = test_data.pid
for track_id in test_data.pid.unique():
    e = signal_true_edges[:, pid[ signal_true_edges[0]] == track_id ]
    true_edges = pid[ e[0]] == pid[e[1]]
    all_true_edges.append( true_edges.sum().numpy() )

    e = pred_edges[:, (pid[ pred_edges[0]] == track_id) |(pid[ pred_edges[1]] == track_id)  ]
    true_edges = pid[ e[0]] == pid[e[1]]
    all_pred_edges.append( e.shape[1] )

histogram = np.histogram(all_true_edges, bins=20, range=(0,20))
max_edge = 700
pred_histogram = np.histogram(all_pred_edges, bins=max_edge, range=(0,max_edge))

true_histogram = pd.DataFrame(
    dict(
        low = histogram[1][:-1],
        high = histogram[1][1:],
        true_hist= histogram[0],
    )
)

pred_histogram = pd.DataFrame(
    dict(
        low = pred_histogram[1][:-1],
        high = pred_histogram[1][1:],
        pred_hist = pred_histogram[0]
    )
)


p1 =  figure(title='Histogram of truth edge number', x_axis_label='Edges', y_axis_label='Count', height=800, width=800) 
p2 =  figure(title='Histogram of predicted edge number', x_axis_label='Edges', y_axis_label='Count', height=800, width=800) 
p1.quad(bottom=0, top='true_hist', left='low', right='high', source=ColumnDataSource(true_histogram))
p2.quad(bottom=0, top='pred_hist', left='low', right='high', source=ColumnDataSource(pred_histogram))
show(row([p1,p2]))

# 2. Construct graphs from metric learning inference

This step performs model inference on the entire input datasets (train, validation and test), to obtain input graphs to the graph neural network.

In [82]:
graph_builder = run_metric_learning_inference(CONFIG)

INFO:-------------------- Step 2: Constructing graphs from metric learning model  --------------------
INFO:-------------------- a) Loading trained model --------------------
INFO:-------------------- b) Running inferencing --------------------
INFO:Loaded event: /global/cfs/cdirs/m3443/data/trackml-codalab/train_all/event000021000
INFO:Loaded event: /global/cfs/cdirs/m3443/data/trackml-codalab/train_all/event000021001
INFO:Loaded event: /global/cfs/cdirs/m3443/data/trackml-codalab/train_all/event000021002


Training finished, running inference to build graphs...


100%|██████████| 10/10 [00:03<00:00,  2.55it/s]


# 3. Train graph neural networks

In [83]:
gnn_trainer, gnn_model = train_gnn(CONFIG)

INFO:--------------------  Step 3: Running GNN training  --------------------
INFO:-------------------- a) Initialising model --------------------
INFO:-------------------- b) Running training --------------------
INFO:GPU available: True, used: True
INFO:TPU available: False, using: 0 TPU cores
INFO:IPU available: False, using: 0 IPUs
INFO:HPU available: False, using: 0 HPUs
INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:
  | Name                   | Type       | Params
------------------------------------------------------
0 | node_encoder           | Sequential | 35.1 K
1 | edge_encoder           | Sequential | 66.4 K
2 | edge_network           | Sequential | 82.8 K
3 | node_network           | Sequential | 82.8 K
4 | output_edge_classifier | Sequential | 83.2 K
------------------------------------------------------
350 K     Trainable params
0         Non-trainable params
350 K     Total params
1.401     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/global/cfs/cdirs/m3443/usr/pmtuan/conda/exatrkx-hsf/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


65.7394910593215
65.7394910593215
65.7394910593215
65.7394910593215
65.7394910593215
65.7394910593215
65.7394910593215
65.7394910593215
52.250693837204885
52.250693837204885
52.250693837204885
52.250693837204885
52.250693837204885
52.250693837204885
52.250693837204885
52.250693837204885


/global/cfs/cdirs/m3443/usr/pmtuan/conda/exatrkx-hsf/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


...

/global/cfs/cdirs/m3443/usr/pmtuan/conda/exatrkx-hsf/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/global/cfs/cdirs/m3443/usr/pmtuan/conda/exatrkx-hsf/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/global/cfs/cdirs/m3443/usr/pmtuan/conda/exatrkx-hsf/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


22.75327590180833
22.743493195618075
22.75327590180833
22.743493195618075
22.75327590180833
22.743493195618075
22.75327590180833
22.743493195618075
18.61304287192095
18.61304287192095
18.61304287192095
18.61304287192095
18.61304287192095
18.61304287192095
18.61304287192095
18.61304287192095
17.762469339530178
17.762469339530178
17.762469339530178
17.762469339530178
17.762469339530178
17.762469339530178
17.762469339530178
17.762469339530178


/global/cfs/cdirs/m3443/usr/pmtuan/conda/exatrkx-hsf/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/global/cfs/cdirs/m3443/usr/pmtuan/conda/exatrkx-hsf/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/global/cfs/cdirs/m3443/usr/pmtuan/conda/exatrkx-hsf/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/global/cfs/cdirs/m3443/usr/pmtuan/conda/exatrkx-hsf/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. G

15.490077774941682
15.490077774941682
15.490077774941682
15.490077774941682
15.490077774941682
15.490077774941682
15.490077774941682
15.490077774941682
15.315032875840101
15.339186207500907
15.315032875840101
15.315032875840101
15.363727934444702
15.363727934444702
15.363727934444702
15.363727934444702
19.24041049011965
19.24041049011965
19.24041049011965
19.24041049011965
19.24041049011965
19.24041049011965
19.24041049011965
19.24041049011965
28.92857884102605
28.950765921566727
28.956737499042667
28.956737499042667
28.956737499042667
28.956737499042667
28.956737499042667
28.956737499042667


INFO:-------------------- c) Saving model --------------------


# Step 4: GNN inference 

In [84]:
run_gnn_inference(CONFIG)

INFO:-------------------- Step 4: Scoring graph edges using GNN  --------------------
INFO:-------------------- a) Loading trained model --------------------
INFO:-------------------- b) Running inferencing --------------------


Training finished, running inference to filter graphs...
Building train


100%|██████████| 80/80 [00:00<00:00, 774.39it/s]


Building val


  0%|          | 0/10 [00:00<?, ?it/s]/global/cfs/cdirs/m3443/usr/pmtuan/conda/exatrkx-hsf/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


...

  0%|          | 0/10 [00:00<?, ?it/s]

15.863337779245587
15.863337779245587
15.863337779245587
15.863337779245587
15.863337779245587
15.863337779245587
15.863337779245587
15.863337779245587
15.56282906638634
15.56282906638634
15.56282906638634
15.56282906638634
15.56282906638634
15.56282906638634
15.582333792003888
15.56282906638634


 70%|███████   | 7/10 [00:00<00:00, 44.69it/s]

29.00719247377331
29.02836001669614
29.02836001669614
29.02836001669614
29.02836001669614
29.02836001669614
29.02836001669614
29.02836001669614
16.1033175147217
16.1033175147217
16.1033175147217
16.087102011656956
16.087102011656956
16.1033175147217
16.087102011656956
16.087102011656956
16.169575322528903
16.169575322528903
16.169575322528903
16.166686632611682
16.166686632611682
16.169575322528903
16.166686632611682
16.166686632611682
25.802785740892524
25.802785740892524
25.802785740892524
25.802785740892524
25.802785740892524
25.802785740892524
25.802785740892524
25.802785740892524


100%|██████████| 10/10 [00:00<00:00, 36.52it/s]


# Step 5: Build track candidates from GNN

In [2]:
build_track_candidates(CONFIG)

INFO:--------------------  Step 5: Building track candidates from the scored graph  --------------------
INFO:-------------------- a) Loading scored graphs --------------------
INFO:-------------------- b) Labelling graph nodes --------------------


  0%|          | 0/114 [00:00<?, ?it/s]